In [ ]:
!pip install opencv-python numpy scikit-image tqdm matplotlib

In [ ]:
import cv2
import numpy as np
import os
from google.colab import files
from skimage.metrics import structural_similarity as ssim
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

# Importar o módulo QeLE
from src.qe_core import QeEngine

In [ ]:
# Carregar imagem de exemplo
img_path = "examples/sample_images/IM3.png"
img_bgr = cv2.imread(img_path)
if img_bgr is None:
    raise FileNotFoundError(f"Imagem {img_path} não encontrada.")

img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
h, w = img_rgb.shape[:2]
print(f"Imagem carregada: {w}x{h} pixels")

In [ ]:
# Criar instância do QeEngine
qe_engine = QeEngine()

# Gerar arquivo Qe
qe_text, h, w = qe_engine.image_to_qe_text(img_path)
with open("image.qe", "w") as f:
    f.write(qe_text)

print("Arquivo Qe gerado: image.qe")

In [ ]:
# Reconstruir imagem a partir do Qe
with open("image.qe", "r") as f:
    qe_text = f.read()

img_rec = qe_engine.qe_text_to_image(qe_text)
img_rec_bgr = cv2.cvtColor(img_rec, cv2.COLOR_RGB2BGR)
cv2.imwrite("reconstructed.png", img_rec_bgr)

print("Imagem reconstruída salva como 'reconstructed.png'")

In [ ]:
# Comparar imagens original e reconstruída
ssim_score, _ = ssim(img_rgb, img_rec, channel_axis=2, data_range=255)
print(f"SSIM: {ssim_score:.6f}")

plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.imshow(img_rgb)
plt.title("Original")
plt.axis('off')
plt.subplot(1, 2, 2)
plt.imshow(img_rec)
plt.title("Reconstruída")
plt.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
# Relatório de tamanhos
def get_file_size_mb(path):
    if os.path.exists(path):
        return os.path.getsize(path) / (1024 * 1024)
    return 0.0

size_original_mb = get_file_size_mb(img_path)
size_reconstructed_mb = get_file_size_mb("reconstructed.png")
size_qe_mb = get_file_size_mb("image.qe")

print("\n[📊] RELATÓRIO DE DADOS")
print(f"Imagem Base:               {size_original_mb:.2f} MB")
print(f"Img Processada (reconstr):  {size_reconstructed_mb:.2f} MB")
print(f"Arquivo Qe:                {size_qe_mb:.2f} MB")
print(f"Img Reconstruida:           {size_reconstructed_mb:.2f} MB")
print(f"Img Comparação:             {get_file_size_mb('comparison.png'):.2f} MB")

In [ ]:
# Baixar arquivos visuais
files.download("reconstructed.png")
files.download("image.qe")  # opcional
plt.savefig("comparison.png", dpi=150, bbox_inches='tight')
files.download("comparison.png")

print("\n[✅] Sistema Qe concluído! Arquivo .qe gerado, mas NÃO baixado.")